rri2csvbot


In [2]:
import os
import shutil
import pyautogui
import random
import time
import math
import pyperclip
import pandas as pd
# Function to convert .rrl files to .csv (copy to logs folder)
def convert_rrl_to_csv(loco_log, logs):
    try:
        for file_name in os.listdir(loco_log):
            if file_name.endswith('.rrl'):
                loco_log_path = os.path.join(loco_log, file_name)
                dest_path = os.path.join(logs, file_name)
                shutil.copy(loco_log_path, dest_path)
        print("Copied .rrl files from loco log to logs folder successfully.")
    except Exception as e:
        print(f"An error occurred while copying files: {e}")

# Function to move the mouse stubbornly to a target position
def move_mouse_stubbornly(target_x, target_y, max_attempts=20, tolerance=2):
    for attempt in range(max_attempts):
        current_x, current_y = pyautogui.position()
        distance = math.hypot(target_x - current_x, target_y - current_y)
        if distance <= tolerance:
            break  # We're close enough
        pyautogui.moveTo(target_x, target_y, duration=random.uniform(0.1, 0.3))
        time.sleep(0.05)
    else:
        print("Warning: Mouse did not reach the target precisely.")

# Function to click the taskbar icon
def click_taskbar_icon(icon_number=5):
    screen_width, screen_height = pyautogui.size()

    taskbar_height = 40
    icon_width = 80
    num_icons = 10
    taskbar_y = screen_height - taskbar_height

    total_icons_width = num_icons * icon_width
    start_x = (screen_width - total_icons_width) // 2

    icon_x = start_x + (icon_number - 1) * icon_width + icon_width // 2

    print(f"Clicking the {icon_number}th icon on the taskbar at ({icon_x}, {taskbar_y})")

    move_mouse_stubbornly(icon_x, taskbar_y)
    time.sleep(random.uniform(0.1, 0.2))
    pyautogui.click(icon_x, taskbar_y)

# Function to open a file with the browse option
def open_file_with_browse(folder_path, file_name):
    # Construct the file path
    file_path = os.path.join(folder_path, file_name)
    if os.path.exists(file_path):
        # Copy the file path to the clipboard
        pyperclip.copy(file_path)
        print(f"File path copied to clipboard: {file_path}")
        # Simulate pasting and opening the file path in a browsing interface
        time.sleep(2)  # Wait for the browsing dialog to open
        pyautogui.hotkey('ctrl', 'v')  # Paste the file path
        time.sleep(1)  # Wait for paste action
        pyautogui.press('enter')  # Press Enter to open the file
        print("File path pasted and file opened!")
    else:
        print(f"File '{file_name}' not found in the folder '{folder_path}'.")

# Main code for processing multiple loco_log folders
parent_source_folder = r"C:\Users\hbl\Desktop\automation scripts\rri2csvbot_v1\logscvrt"  # loco logs folder
logs = r"D:\TCAS_Utilities\v4.0\MaintenanceTools_V4.0\Log"  # Logs folder
final_destination_base = r"C:\Users\hbl\Downloads\ProcessedCSVs"  # Final destination base folder

# List of specific CSV files to move
target_files = [
    "LocoRadioRegStringLog.csv",
    "LocoRadioStringLog.csv",
    "StnRegLogV4.csv",
    "StnRespStringLog.csv"
]

# Iterate through all loco_log folders in the parent folder
for loco_log_folder in os.listdir(parent_source_folder):
    loco_log_path = os.path.join(parent_source_folder, loco_log_folder)

    # Ensure it is a directory (loco_log folder)
    if os.path.isdir(loco_log_path):
        print(f"Processing folder: {loco_log_folder}")

        # 1. Copy .rrl files to the logs folder
        convert_rrl_to_csv(loco_log_path, logs)

        # 2. Perform automation tasks (clicking, moving mouse, etc.)
        click_taskbar_icon(5)
        time.sleep(5)
        x, y = 805, 320
        pyautogui.moveTo(x, y, duration=0.01)
        pyautogui.click()
        time.sleep(0.5)
        print("login ascii checked")

        # Export and read
        x, y = 1100, 770
        pyautogui.moveTo(x, y, duration=0.01)
        pyautogui.click()
        time.sleep(6)
        print("export button clicked")
        x, y = 180, 770
        pyautogui.moveTo(x, y, duration=0.01)
        pyautogui.click()
        print('readfile clicked')

        # Open the file with browse
        open_file_with_browse(logs, "combined.rrl")
        time.sleep(45)
        x, y = 1100, 770
        pyautogui.moveTo(x, y, duration=0.01)
        pyautogui.click()
        time.sleep(8)

        # 3. Exit
        print("export button clicked for 2nd time")
        pyautogui.hotkey('alt', 'f4')
        time.sleep(3)

        # 4. Move specific CSV files to a new folder
        new_folder = os.path.join(logs, 'csv_files')
        os.makedirs(new_folder, exist_ok=True)

        # Move the specific CSV files to the new folder
        for filename in target_files:
            source_file = os.path.join(logs, filename)
            if os.path.exists(source_file):
                shutil.move(source_file, os.path.join(new_folder, filename))

        # Ensure the final destination directory exists
        final_destination = loco_log_path
        os.makedirs(final_destination, exist_ok=True)

        # Move the csv_files folder to the final destination
        shutil.move(new_folder, os.path.join(final_destination, 'csv_files'))

        print(f"Selected CSV files moved and 'csv_files' folder from '{loco_log_folder}' relocated successfully!")

        # 5. Delete all remaining files in the log folder
        for file in os.listdir(logs):
            file_path = os.path.join(logs, file)
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")

        print(f"All remaining files and folders in '{logs}' deleted.")
        time.sleep(2)
print("Process completed for all 'loco_log' folders!")
for subdir, _, _ in os.walk(parent_source_folder):
    csv_folder = os.path.join(subdir, "csv_files")
    if os.path.exists(csv_folder) and os.path.isdir(csv_folder):
        for file in os.listdir(csv_folder):
            if file.endswith(".csv"):
                file_path = os.path.join(csv_folder, file)
                
                # Read 
                df = pd.read_csv(file_path)
                
                # Sort by 'FrameNo' 
                df_sorted = df.sort_values(by='FrameNo')
                
                #overwrite original
                df_sorted.to_csv(file_path, index=False)
                
                print(f"Sorted {file_path} by FrameNo.")

print("Sorting complete!")

Processing folder: 12339_UP _DIRECTION _BIU_COALFIELD EXPRESS _07.07.25
Copied .rrl files from loco log to logs folder successfully.
Clicking the 5th icon on the taskbar at (760, 860)
login ascii checked
export button clicked
readfile clicked
File path copied to clipboard: D:\TCAS_Utilities\v4.0\MaintenanceTools_V4.0\Log\combined.rrl
File path pasted and file opened!
export button clicked for 2nd time
Selected CSV files moved and 'csv_files' folder from '12339_UP _DIRECTION _BIU_COALFIELD EXPRESS _07.07.25' relocated successfully!
All remaining files and folders in 'D:\TCAS_Utilities\v4.0\MaintenanceTools_V4.0\Log' deleted.
Process completed for all 'loco_log' folders!
Sorted C:\Users\hbl\Desktop\automation scripts\rri2csvbot_v1\logscvrt\12339_UP _DIRECTION _BIU_COALFIELD EXPRESS _07.07.25\csv_files\LocoRadioRegStringLog.csv by FrameNo.
Sorted C:\Users\hbl\Desktop\automation scripts\rri2csvbot_v1\logscvrt\12339_UP _DIRECTION _BIU_COALFIELD EXPRESS _07.07.25\csv_files\LocoRadioStringLog